# PostgreSQL Practice

This notebook will contain the work associated with postgresqltutorial.com's python tutorial

Source: http://www.postgresqltutorial.com/postgresql-python/

## Step 1: Create and connect to the 'suppliers' database in terminal

Enter `/Applications/Postgres.app/Contents/Versions/12/bin/psql -p5432 "database_name"` where `database_name` is any existing database or the psql command if you already configured it

In [10]:
# Steps adapted from https://stackoverflow.com/a/46877364

# Install psycopg2 if not already on your computer

# Step 1: Attempt to run the following in command line (Python 3):
# sudo apt install libpq-dev python3-dev

# A Java pop-up request may tell you:
# "To use the “apt” command-line tool you need to install a JDK." (Java Developer Kit)
# This is because this package is written in C and adapted for python

# You have a few options for each operating system compressed and uncompressed,
# I selected jdk-13.0.1_osx-x64_bin.dmg
# Run the sudo command after installation 



!pip install psycopg2

# If you are still having trouble try updating setup tools
# !pip install --upgrade pip setuptools

sudo -H pip3 install gitsome

  Using cached https://files.pythonhosted.org/packages/84/d7/6a93c99b5ba4d4d22daa3928b983cec66df4536ca50b22ce5dcac65e4e71/psycopg2-2.8.4.tar.gz
    ERROR: Command errored out with exit status 1:
     command: /anaconda3/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/mf/x1x3gjq118g2js0wpqqbmvs00000gn/T/pip-install-i38r8zhj/psycopg2/setup.py'"'"'; __file__='"'"'/private/var/folders/mf/x1x3gjq118g2js0wpqqbmvs00000gn/T/pip-install-i38r8zhj/psycopg2/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/mf/x1x3gjq118g2js0wpqqbmvs00000gn/T/pip-install-i38r8zhj/psycopg2/pip-egg-info
         cwd: /private/var/folders/mf/x1x3gjq118g2js0wpqqbmvs00000gn/T/pip-install-i38r8zhj/psycopg2/
    Complete output (23 lines):
    running egg_info
    creating /private/var/folders/mf/x1x3gjq118g2js0wpqqbmv

In [9]:
import psycopg2
from config import config

ModuleNotFoundError: No module named 'psycopg2'

In [2]:
#Connect in python to the database by creating a cursor object

conn = psycopg2.connect("dbname=suppliers user=postgres password=postgres")

NameError: name 'psycopg2' is not defined